<h1>4. 우리기업의 vip 고객은 누구인가?</h1>

이번 노드 우리 기업의 VIP는 누구인가? 에서는 기술통계량을 중심으로 빈도분석을 수행합니다.
그리고 빈도분석에 기반한 마케팅 기법인 RFM(Recency, Frequency, Monetary)분석을 통해
우리 기업의 VIP고객이 누구인지를 밝혀보도록 하겠습니다.

먼저 Pandas, Numpy, Matplotlib, Seaborn을 호출하고, sales_05_8.csv에 저장된 데이터를 확인해 봅시다.

In [1]:
# Pandas, Numpy, Matplotlib, Seaborn을 호출
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
import seaborn as sns
import datetime as dt

# sales_data 변수를 만들고 sales_05.csv 값을 할당
sales_data = pd.read_csv('~/aiffel/data_analysis_basic/data/sales_05_8.csv')

# sales_data의 head 값 확인하기
sales_data.head()

,Install,Time_stamp,Status,ID,Payment,Game_Name
0,2022-01-14,2022-01-27,purchase,Heidi_635,19900,Soccer Manager
1,2022-03-10,2022-03-13,purchase,Courtney_067,900,Mine Ground
2,2022-02-13,2022-02-19,purchase,David_737,9900,Soccer Manager
3,2022-02-07,2022-02-25,purchase,Lindsay_820,4900,Mine Ground
4,2022-03-06,2022-03-10,purchase,Jon_823,49900,Soccer Manager


In [2]:
# sales_data의 정보 확인하기
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75087 entries, 0 to 75086
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Install     75087 non-null  object
 1   Time_stamp  75087 non-null  object
 2   Status      75087 non-null  object
 3   ID          75087 non-null  object
 4   Payment     75087 non-null  int64 
 5   Game_Name   75087 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.4+ MB


- 고유한 ID는 과연 몇 개나 있을지 한번 확인해보겠습니다.
- 데이터의 고유값을 보여주는 unique() 함수와 데이터의 개수를 알려주는 len() 함수를 사용하면 쉽게 확인할 수 있습니다.

In [3]:
# salses_data의 ID의 고유값 확인하기
len(sales_data['ID'].unique())

52672

총 52,672명의 고객이 지금까지 우리 게임을 즐기고 있었습니다.<br>
이 분들 중에서 우리들은 VIP를 어떻게 가려낼 수 있을까요?<br>
일반적으로 사용되는 방법은 RFM 입니다.<br>
RFM은 Recency, Frequency, Monetary 의 약자로<br>
가장 최근에 구매한 고객, 가장 빈번하게 구매한 고객, 
가장 많은 금액을 구매한 고객을 기준으로 고객에게 등급을 부여하는 방식입니다.<br>
구체적으로 RFM의 각 요소는 아래와 같이 계산할 수 있습니다.<br>

- 최근성(Recency): 현재 시점에서 마지막 기록 간의 차이
- 빈번함(Frequency): 지금까지 결제한 총 횟수
- 결제금액(Monetary): 지금까지 결제한 총 금액
- RFM을 통해 산출되는 RFM Score를 통해 우리 기업의 VIP를 정의할 수 있습니다.

지금부터 RFM을 적용해보도록 하겠습니다.
분석을 하고 있는 현재 날짜를 2022년 6월 10일로 가정해 봅시다.

<h4>RFM 중 R과 F, 즉 "시간(Time)"계산을 위해 "datetime" 라이브러리를 사용하고, <br>
M을 계산하기 위해 "Status"가 "Purchase인" 행들만 골라냅니다.</h4>

In [8]:
# sales_data에 Now열을 만들고 2022년 6월 10일로 가정
sales_data['Now'] = '2022-06-10'
# sales_data

# Now열 값을 시간 계산을 위한 datetime으로 전환
sales_data['Now'] = pd.to_datetime(sales_data['Now'])

# sales_data의 Time_stamp열 값을 시간 계산을 위한 datetime으로 전환
sales_data['Time_stamp'] = pd.to_datetime(sales_data['Time_stamp'])

# sales_data 중 Status가 'Purchase'인 행들만 골라내기
sales_data = sales_data[sales_data['Status']=='purchase']

sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27682 entries, 0 to 27681
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Install     27682 non-null  object        
 1   Time_stamp  27682 non-null  datetime64[ns]
 2   Status      27682 non-null  object        
 3   ID          27682 non-null  object        
 4   Payment     27682 non-null  int64         
 5   Game_Name   27682 non-null  object        
 6   Now         27682 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 1.7+ MB


/tmp/ipykernel_31/605186962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['Now'] = '2022-06-10'
/tmp/ipykernel_31/605186962.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['Now'] = pd.to_datetime(sales_data['Now'])
/tmp/ipykernel_31/605186962.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

총 75,087건의 로그(Log)기록 중 결제(Purchase) 기록은 27,682건이네요.<br>
플레이 수 대비 약 36% 수준이 결제를 했다니 나쁘지 않은 것 같습니다.

이제 결제한 기록이 있는 행만 빼서 RFM 즉, 최근성과 빈번성, 결제금액을 도출해보겠습니다.

<h4>먼저 결제한 기록이 있는 행만 얻기 위해 sales_data에 Interval 열을 만들고,<br> 지금 시간에서 마지막 기록된 시간인 Time_stamp 값을 뺍니다.</h4>

In [9]:
# sales_data에 'Interval'열을 만들고 지금 시간에서 마지막 기록된 시간인 Time_stamp 값 빼기
sales_data['Interval'] = sales_data['Now'] - sales_data['Time_stamp']

In [11]:
sales_data

,Install,Time_stamp,Status,ID,Payment,Game_Name,Now,Interval
0,2022-01-14,2022-01-27,purchase,Heidi_635,19900,Soccer Manager,2022-06-10,134 days
1,2022-03-10,2022-03-13,purchase,Courtney_067,900,Mine Ground,2022-06-10,89 days
2,2022-02-13,2022-02-19,purchase,David_737,9900,Soccer Manager,2022-06-10,111 days
3,2022-02-07,2022-02-25,purchase,Lindsay_820,4900,Mine Ground,2022-06-10,105 days
4,2022-03-06,2022-03-10,purchase,Jon_823,49900,Soccer Manager,2022-06-10,92 days
...,...,...,...,...,...,...,...,...
27677,2022-05-11,2022-05-30,purchase,Debra_720,900,Mine Ground,2022-06-10,11 days
27678,2022-04-26,2022-04-28,purchase,Kevin_404,900,Soccer Manager,2022-06-10,43 days
27679,2022-03-30,2022-04-09,purchase,Roberto_426,9900,Mine Ground,2022-06-10,62 days
27680,2022-05-09,2022-05-27,purchase,Craig_278,4900,Mine Ground,2022-06-10,14 days


<h3>새로운 데이터프레임 RFM_data를 만들고,<br> 
groupby() 함수를 이용해서 sales_data 데이터프레임의 "Recency, Frequency, Monetary" 값을 집계하여 RFM_data 데이터프레임에 추가합니다.<br>

그 후 RFM_data의 열 값을 각각 Recency, Frequency, Monetary로 변경합니다.</h3>
최근성(Recency): 현재 시점에서 마지막 기록 간의 차이<br>
빈번함(Frequency): 지금까지 결제한 총 횟수<br>
결제금액(Monetary): 지금까지 결제한 총 금액<br>


<h4>열 값 변경에 사용하는 함수는 agg()(aggregate) 입니다.<br>
agg()함수는 연산 값을 여러 열에 적용할 때 사용합니다.<br>

즉, 아래 코드는 다음과 같은 의미를 갖습니다.<br>
RFM_data 변수를 만들고, sales_data의 ID 열을 기준으로 집계하는데, 
Interval 열은 min(), ID 열은 count(), Payment 열은 sum()으로 집계해서 RFM_data에 할당한다는 의미입니다</h4>

In [16]:
# RFM_data 변수를 만들고 Groupby()함수를 이용해서 Recency, Frequency, Monetary 집계하기
RFM_data = sales_data.groupby(['ID']).agg({'Interval':'min','ID': 'count', 'Payment': 'sum'})

# RFM_data의 열 값을 각각 Recency, Frequency, Monetary로 변경
RFM_data.columns = ['Recency', 'Frequency', 'Monetary']
RFM_data.head()

,Recency,Frequency,Monetary
ID,,,
Aaron_027,60 days,5,40500
Aaron_031,102 days,5,17500
Aaron_077,40 days,4,26600
Aaron_165,21 days,4,49600
Aaron_224,108 days,1,900


In [19]:
# 마지막으로 RFM_data의 Recency 열 값을 숫자로 변경합니다. (dt.days)
# 총 5,267명의 결제 기록이 있는 고객들을 대상으로 RFM 요소를 계산했습니다.

RFM_data['Recency'] = RFM_data['Recency'].dt.days
RFM_data

AttributeError: Can only use .dt accessor with datetimelike values

In [20]:
# RFM의 기술통계랑 살펴보기
RFM_data.describe()

,Recency,Frequency,Monetary
count,5267.000000,5267.000000,5267.000000
mean,66.517752,5.255743,50340.003797
std,35.684775,6.995041,73012.364791
min,7.000000,1.000000,900.000000
25%,36.000000,1.000000,9900.000000
50%,61.000000,5.000000,29900.000000
75%,94.000000,5.000000,55500.000000
max,158.000000,37.000000,582700.000000


- 한 눈에 보기 쉽게 테이블을 변경해 봅시다.

대문자 T는 표시되는 테이블을 전치(transpose)하라는 뜻, 즉 '왼쪽으로 90° 돌려서 표시해줘' 라는 의미입니다. <br>
함수 round(2)는 결과 값의 소수 둘째자리까지 남겨두라는 의미입니다. 즉 소수 셋째자리에서 반올림합니다.

In [21]:
# RFM_data의 기술통계량 살펴보기 
RFM_data.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
Recency,5267.0,66.52,35.68,7.0,36.0,61.0,94.0,158.0
Frequency,5267.0,5.26,7.00,1.0,1.0,5.0,5.0,37.0
Monetary,5267.0,50340.00,73012.36,900.0,9900.0,29900.0,55500.0,582700.0


RFM의 각 요소별 기술통계량이 도출되었습니다. 

---
1). **count: 데이터 개수**
결제기록이 있는 고객의 수는 총 5,267명입니다.

2). **mean: 각 요소별 평균**
데이터 값의 총합을 개수로 나눈 값입니다.<br>
RFM처럼 측정단위가 다른 데이터를 다룰 경우 단위를 잘 고려해야 합니다.<br>

Recency는 66.5일, Frequency는 5.2회, Monetary는 50,340원입니다.<br>

3). **std(Standard Deviation): 각 요소별 표준편차**
표준편차는 평균을 중심으로 각 요소 값들이 얼마나 떨어져 있는지를 나타냅니다.
분산의 제곱근으로 계산될 수 있습니다.
편차를 제곱함으로써 단위 의미가 없어지는 분산에 비해 각 측정단위를 갖는다는 장점이 있습니다.

Recency는 35.68일, Frequency는 7회, Monetary는 73,012원입니다.

4).**min: 각 요소별 최솟값**

Recency는 7일, Frequency는 1회, Monetary는 900원입니다.
가장 최근에 결제한 고객은 7일전에 결제를 했네요.
25%~75%: 사분위수 범위

---
전체 데이터 집단을 크기순으로 나열했을 때, 전체의 25% 위치의 값, 50% 위치의 값, 75% 위치의 값을 의미합니다.<br>
이때 50% 위치의 값을 중위값(Median)이라고도 합니다.<br>
25% 위치에서부터 75% 위치까지의 범위를 사분위범위(IQR:Interquartile Range)라고도 합니다.<br>
표준편차가 75% 위치값을 벗어나는 경우 데이터가 넓게 흩어져있다는 의미가 됩니다.<br>

Recency를 제외하고 Frequency와 Monetary의 데이터는 넓게 흩어져 있을 것 같습니다.
max: 각 요소별 최댓값

Recency는 158일, Frequency는 37회, Monetary는 582,700원입니다.<br>
가장 자주 결제한 고객은 무려 37회나 결제했습니다.<br>
그리고 가장 많은 누적금액을 결제한 고객은 582,700원입니다.<br>
기술통계량을 살펴봤는데요, 시각화를 통해서 조금 더 구체적으로 살펴보겠습니다.

<h3>Recency, Frequency, Monetary의 순서로 각각 시각화해보겠습니다.</h3>

우선 Recency를 기준으로 잡고, 데이터 RFM_data를 내림차순으로 정렬하겠습니다. <br>
sort_values() 함수를 사용하면 데이터를 정렬할 수 있습니다. <br>
by에는 기준을 넣고, ascending을 True와 False로 지정해 오름차순과 내림차순으로 정렬할 수 있어요.

In [22]:
# RFM_data의 Recency열 값을 내림차순(높은값에서 낮은 값 순서로)으로 정렬
temp_var = RFM_data.sort_values(by=['Recency'], ascending=False)
temp_var.head()

,Recency,Frequency,Monetary
ID,,,
Nancy_218,158,1,900
Michael_254,155,1,900
Craig_831,155,1,19900
Matthew_192,154,1,900
Greg_925,154,1,900
